In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

In [23]:
train = pd.read_csv("Cleaned_dataset.csv")

In [109]:
class training_dataset(Dataset):
    def __init__(self, target_transform, feature_transform):
        self.raw_data=train
        self.target_transform=target_transform
        self.feature_transform=feature_transform
    def __len__(self):
        return len(self.raw_data.index)
    def __getitem__(self, idx):
        row=(self.raw_data.iloc[idx])
        features=torch.tensor(row[1: -1])
        target=torch.tensor(row[-1])
        return self.feature_transform(features),self.target_transform(target)

In [110]:
trainingData = training_dataset(lambda x:torch.tensor([0,1]).double() if x.item()==1 else torch.tensor([1,0]).double(), lambda x:x.float())

In [114]:
trainingData.__getitem__(3)

(tensor([0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.1058, 0.2830, 0.3243, 1.0000,
         1.0000, 1.0000]),
 tensor([0., 1.], dtype=torch.float64))

In [115]:
trainingData=DataLoader(trainingData, batch_size=16, shuffle=True)

In [126]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.linear_stack=nn.Sequential(
            nn.Linear(11,100),
            nn.Sigmoid(),
            nn.BatchNorm1d(100),
            nn.Linear(100,200),
            
            nn.Sigmoid(),
            nn.BatchNorm1d(200),
            nn.Linear(200,300),
            
            nn.Sigmoid(),
            nn.BatchNorm1d(300),
            nn.Linear(300,300),
            nn.Sigmoid(),
            nn.BatchNorm1d(300),
            nn.Linear(300,200),
            
            nn.Sigmoid(),
            nn.BatchNorm1d(200),
            nn.Linear(200,50),
            nn.Sigmoid(),
            nn.BatchNorm1d(50),
            nn.Linear(50,2),
            nn.Sigmoid()
        )
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        return self.softmax(self.linear_stack(x)).float()

In [145]:
class reg(nn.Module):
    def __init__(self):
        super(reg,self).__init__()
        self.linear1=nn.Linear(11,2)
        self.sig=nn.Sigmoid()
    def forward(self, x):
        x=self.linear1(x)
        x=self.sig(x)
        return x.float()

In [146]:
#model = Network()
model=reg()

In [147]:
model(next(iter(trainingData))[0])

tensor([[0.6848, 0.4351],
        [0.7406, 0.4010],
        [0.7202, 0.3931],
        [0.7124, 0.5029],
        [0.7139, 0.4764],
        [0.6825, 0.3383],
        [0.7967, 0.4083],
        [0.6734, 0.4905],
        [0.8060, 0.4777],
        [0.5947, 0.5591],
        [0.7403, 0.6324],
        [0.7925, 0.4623],
        [0.7259, 0.5210],
        [0.6704, 0.4619],
        [0.7721, 0.3882],
        [0.5785, 0.5193]], grad_fn=<SigmoidBackward0>)

In [148]:
learning_rate=0.00002
batch_size=16
epochs=300
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [149]:

def training_loop(model, optimizer, loss_fn, dataloader, batch_size):
    length=len(dataloader.dataset)
    count=0
    for batch, (x,y) in enumerate(dataloader):
        output=model(x)
        
        loss = loss_fn(output, y.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count+=1
        if count%30==0:
            print(f"Batch: {batch}  Loss:{loss.item()}")
            #print(f"Predicted: {output[0]}   Actual: {y[0]}  Loss: {loss_fn(output[0], y[0]).item()}")

In [150]:
for name,param in model.named_parameters():
    if param.requires_grad==True:
        print (name,param)

linear1.weight Parameter containing:
tensor([[-0.0982,  0.1632, -0.2389,  0.2962,  0.2236, -0.0232,  0.2260,  0.2161,
          0.2964,  0.2817, -0.2948],
        [-0.1189, -0.0450,  0.1509, -0.1579, -0.2901,  0.0890,  0.0500,  0.0997,
         -0.1008,  0.2701, -0.2542]], requires_grad=True)
linear1.bias Parameter containing:
tensor([0.1105, 0.0738], requires_grad=True)


In [151]:
for i in range(epochs):
    training_loop(model, optimizer, loss_fn,trainingData,32)

Batch: 29  Loss:0.7169330716133118
Batch: 29  Loss:0.7920764684677124
Batch: 29  Loss:0.7827703356742859
Batch: 29  Loss:0.7228395342826843
Batch: 29  Loss:0.7738779783248901
Batch: 29  Loss:0.7930233478546143
Batch: 29  Loss:0.7535129189491272
Batch: 29  Loss:0.7329697012901306
Batch: 29  Loss:0.7684309482574463
Batch: 29  Loss:0.8112479448318481
Batch: 29  Loss:0.7042197585105896
Batch: 29  Loss:0.7332278490066528
Batch: 29  Loss:0.7352681756019592
Batch: 29  Loss:0.7873897552490234
Batch: 29  Loss:0.7251299619674683
Batch: 29  Loss:0.7471441626548767
Batch: 29  Loss:0.7095048427581787
Batch: 29  Loss:0.7696402072906494
Batch: 29  Loss:0.7486487627029419
Batch: 29  Loss:0.7143991589546204
Batch: 29  Loss:0.7250944375991821
Batch: 29  Loss:0.7788088321685791
Batch: 29  Loss:0.7415801882743835
Batch: 29  Loss:0.7010969519615173
Batch: 29  Loss:0.77652907371521
Batch: 29  Loss:0.7257486581802368
Batch: 29  Loss:0.7243849635124207
Batch: 29  Loss:0.7839659452438354
Batch: 29  Loss:0.7079

KeyboardInterrupt: 